# FIT5196 Assessment 3
# Task 2 Generate Sparse Representations 
#### Student Name: Syed Ali Alim Rizvi
#### Student ID: 28984773

Date: 3/06/2018

Version: 3.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* from bs4 BeautifulSoup
* re 
* os
* pandas


## 1. Introduction
This assignment comprises the creation of vocabulary, and sparse represention of the files in task 1 as well as it segment and topic boundary boolean representation. 

Tasks:
1. Importing libraries
2. Reading data in
3. Creating the files

More details for each task will be given in the following sections.

## 2. Imports

In [46]:
import re
import os
import pandas as pd
from nltk.tokenize import RegexpTokenizer 
from collections import Counter #for counting elements in list

## 3. Tokenizing files and creating vocab

#### Usint a regex tokenizer

In [47]:
#tokenizer regex
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")

#### importing the stopwords file

In [48]:
#stopwords file
stop_words_file = open('stopwords_en.txt', 'r')
stop_words = stop_words_file.read()
stop_words = stop_words.split('\n')
stop_words_file.close()

#stop words set
stop_words_set = set(stop_words)

#### Tokens
creating unique tokens for words with lengths >=3 for each file and then removing the stop words and the words which occur in more than 132 documents to create vocab

In [49]:
token_dict = {}

#folder to check
xml_file_path = './txt_files'

#all topic files
files = os.listdir(xml_file_path)

#for each topic file 
for xfile in files:
    
    #skip unrequired files
    if xfile == 'example_output.txt':
        continue
        
    topic_id = xfile[:-4]
    
    #create the path to file
    xfile = os.path.join(xml_file_path, xfile)
    
    #if the file is a correct file and is .xml
    if os.path.isfile(xfile) and xfile.endswith('.txt'): 
        
        #read the raw text in
        my_file = open(xfile, 'r')
        raw_text = my_file.read().lower().strip()
        
        #create unigram tokens
        unigram_tokens = tokenizer.tokenize(raw_text)
        
        #create list counts 
        unigram_tokens_count = Counter(unigram_tokens)
        
        #unique tokens set
        unigram_tokens_set = set(unigram_tokens)
        unigram_tokens_set = set([x for x in unigram_tokens_set if len(x)>2])
        
        #unigram without stopwords
        unigram_tokens_wo_sw = unigram_tokens_set - stop_words_set
        unigram_tokens_wo_sw = list(unigram_tokens_wo_sw)
        
        #store in token dictionary
        token_dict[topic_id] = unigram_tokens_wo_sw  
        
        #close file
        my_file.close()

create on token list from all unique tokens collected and creating a vocabulary

In [50]:
#all tokens
all_tokens_list = []

# store all tokens from all files in one list
for tokens in token_dict.values():
    all_tokens_list.extend(tokens)

In [51]:
#getting counts of tokens
all_tokens_counts = Counter(all_tokens_list)
all_tokens_counts = dict(all_tokens_counts)

In [52]:
#getting only clean tokens
all_tokens_clean = {}
all_tokens_dirty = {}

# for each word and its freq accrose files
for key, value in all_tokens_counts.items():
    # if freq is > 132 add it to the dirty dict
    if value>132:
        all_tokens_dirty[key] = value
    # otherwise add it to the clean dict
    else:
        all_tokens_clean[key] = value

In [53]:
#getting clean token list
clean_tokens = [x for x in all_tokens_clean.keys()]
clean_tokens.sort()

In [54]:
#creating the vocab
vocab_dict = {}
vocab = ''

#ofr each word in the cleaned token list create a dictionary and a string 
for ind, word in enumerate(clean_tokens):
    vocab_dict[word] = ind
    vocab = vocab + '{}:{}\n'.format(word, ind)

In [55]:
# write vocab file using the vocab string
vocab_text_file = open('vocab.txt', 'w')
vocab_text_file.write(vocab)
vocab_text_file.close()

In [56]:
#print for checking
print(len(all_tokens_counts), '--Total Tokens without Stop words')
print(len(all_tokens_dirty), '--Dirty Tokens >132')
print(len(all_tokens_clean), '--Clean Tokens <132')

10318 --Total Tokens without Stop words
13 --Dirty Tokens >132
10305 --Clean Tokens <132


## 4. creating boolean representaton and the sparse files for each topic. 

The output of task1 is used to create this. 

this is because if a root topic ends in the middle of a segment, the first half of the segment would be a one in the boolean and the next would be zero. However if the XML version is used to create the sparse and boolean representation the end of root topic would be missed and hence a 1 will not be shown and then it will not remain consistent with the sparse file. 

Hence for creating the sparse file the output of task 1 is used, i.e. if a segment is broken due to an ending of topic in the middle of it, the broken segments are used to sparse the file and create a boolean representation. 

In [57]:
#store the topic segments boolean representation after preprosessing
topic_seg=''

#folder to check
xml_file_path = './txt_files'

#all topic files
files = os.listdir(xml_file_path)

#for each topic file 
for xfile in files:
    
    #skip unrequired files
    if xfile == 'example_output.txt':
        continue
        
    topic_id = xfile[:-4]
    
    #create the path to file
    nxfile = os.path.join(xml_file_path, xfile)
    
    #if the file is a correct file and is .xml
    if os.path.isfile(nxfile) and nxfile.endswith('.txt'): 
        
        #for segment boolean storage
        segs=''
        
        #string to store output in 
        sparse_text = ''
        
        #read the raw text in
        my_file = open(nxfile, 'r')
        raw_text = my_file.read().lower()
        
        #create unigram tokens
        segments = raw_text.split('\n')
        
        #for each segment in the file
        for segment in segments:
            
            #if the segment is stars put stars in sparse file (they will be removed before writing to file)
            if re.search(r'\*', segment) != None:
                sparse_text = sparse_text + '*'*10 + '\n'
                continue
                                
            #get a list of words
            segment_words = tokenizer.tokenize(segment)
            #segment_words = segment.split(' ') if we wnated to create sparse files using raw text 
            
            #check count of words
            word_counts = dict(Counter(segment_words))
            
            #for each word and its count 
            for word, freq in word_counts.items():
                #if the word is in the vocab store the id of the word and its freq
                if word in list(vocab_dict.keys()):
                    word_ind = vocab_dict[word]
                    sparse_text += '{}:{},'.format(word_ind, freq)
                #otherwise do nothing
                else:
                    pass
            
            #add segment to sparse text (if word was added [:-1] would remove comma else would remove \n that was added)
            sparse_text = sparse_text[:-1] + '\n'
        
        #remove any leading or trailing white spaces
        sparse_text = sparse_text.strip()
        
        #create topic seg file
        segs = re.subn(r'[^\n|*]', '', sparse_text)[0]
        segs = re.subn(r'\n\*{10,10}\n?', '1,', segs)[0]
        segs = re.subn(r'\*{10,10}\n?', '', segs)[0]
        segs = re.subn(r'\n', '0,', segs)[0]
        
        #put in topic_seg
        topic_seg = topic_seg + '{}:{}\n'.format(topic_id, segs[:-1])
        
        #remove stars from sparse file
        sparse_text = re.subn(r'\n\*{10,10}', '', sparse_text)[0]
        sparse_text = re.subn(r'\*{10,10}\n', '', sparse_text)[0]
        
        #create sparse file for topic
        out_file_name = './sparse_files/' + xfile
        my_output_file = open(out_file_name, 'w')
        my_output_file.write(sparse_text.strip())
        my_output_file.close()

#### writing the topic segment file

In [58]:
# creating topics_seg file
topic_seg_file = open('topic_seg.txt', 'w')
topic_seg_file.write(topic_seg.strip())
topic_seg_file.close()

## 5. Summary

- In this case if the topic boundary cut the segment boundary, then to reflect it in the boolean and sparse representation, the output of task1 was used. 
- To create the sparse files, the vocab was applied to the tokenized text and not the raw text.
- Along with the stopwords and words occuring in more than 132 documents, words less than length 3 were also removed. 